In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
!pip install ../input/segmentation-model-pt/segmentation_models_pytorch-0.2.0-py3-none-any.whl --no-index --no-deps

In [ ]:
!pip install ../input/segmentation-model-pt/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4

In [ ]:
!pip install ../input/segmentation-model-pt/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3/

In [ ]:
!pip install ../input/timm-pytorch-image-models/pytorch-image-models-master/

In [ ]:
import timm

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

import torch
from torch import nn
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
DIM = (512,512,3)
BATCH_SIZE = 4

CANDIDATES = [
    {
        'model_path':'../input/siimstudyclfmodels/v303/v303/Fold0_effb7_unetplusplus_v303_dungnb_use_2stage_pretrain_ValidLoss0.733_ValidAcc0.000_ValidAUC0.796_ValidmAP0.366_Ep11.pth'
    },
    {
        'model_path':'../input/siimstudyclfmodels/v303/v303/Fold1_effb7_unetplusplus_v303_dungnb_use_2stage_pretrain_ValidLoss0.722_ValidAcc0.000_ValidAUC0.799_ValidmAP0.362_Ep03.pth'
    },
    {
        'model_path':'../input/siimstudyclfmodels/v303/v303/Fold2_effb7_unetplusplus_v303_dungnb_use_2stage_pretrain_ValidLoss0.695_ValidAcc0.000_ValidAUC0.825_ValidmAP0.382_Ep16.pth'
    },
    {
        'model_path':'../input/siimstudyclfmodels/v303/v303/Fold3_effb7_unetplusplus_v303_dungnb_use_2stage_pretrain_ValidLoss0.687_ValidAcc0.000_ValidAUC0.821_ValidmAP0.380_Ep18.pth'
    },
    {
        'model_path':'../input/siimstudyclfmodels/v303/v303/Fold4_effb7_unetplusplus_v303_dungnb_use_2stage_pretrain_ValidLoss0.718_ValidAcc0.000_ValidAUC0.822_ValidmAP0.401_Ep10.pth'
    },
    
]


In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
fast_sub = False

# if df.shape[0] == 2477:
#     fast_sub = True
#     fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
#                          ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
#                          ['65761e66de9f_image', 'none 1 0 0 1 1'], 
#                          ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
#                        columns=['id', 'PredictionString'])
# else:
#     fast_sub = False


# .dcm to .png

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
split = 'test'
save_dir = f'/kaggle/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'/kaggle/tmp/{split}/study/'
os.makedirs(save_dir, exist_ok=True)
if fast_sub:
    xray = read_xray('../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm')
    im = resize(xray, size=600)  
    study = '00086460a852' + '_study.png'
    im.save(os.path.join(save_dir, study))
    xray = read_xray('../input/siim-covid19-detection/train/000c9c05fd14/e555410bd2cd/51759b5579bc.dcm')
    im = resize(xray, size=600)  
    study = '000c9c05fd14' + '_study.png'
    im.save(os.path.join(save_dir, study))
else:   
    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=600)  
            study = dirname.split('/')[-2] + '_study.png'
            im.save(os.path.join(save_dir, study))


# study predict

In [ ]:
import numpy as np 
import pandas as pd
if fast_sub:
    df = fast_df.copy()
else:
    df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
id_laststr_list  = []
for i in range(df.shape[0]):
    id_laststr_list.append(df.loc[i,'id'][-1])
df['id_last_str'] = id_laststr_list

study_len = df[df['id_last_str'] == 'y'].shape[0]

In [ ]:
df['id_last_str']

In [ ]:
study_len

In [ ]:
# IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)

#load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
if fast_sub:
    sub_df = fast_df.copy()
else:
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
sub_df = sub_df[:study_len]
test_paths = f'/kaggle/tmp/{split}/study/' + sub_df['id'] +'.png'

sub_df['negative'] = 0
sub_df['typical'] = 0
sub_df['indeterminate'] = 0
sub_df['atypical'] = 0


label_cols = sub_df.columns[2:]

sub_df['test_path'] = test_paths

In [ ]:
from typing import Optional, List
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast

from segmentation_models_pytorch.encoders import get_encoder
from segmentation_models_pytorch.unet.decoder import UnetDecoder
from segmentation_models_pytorch.unetplusplus.decoder import UnetPlusPlusDecoder
from segmentation_models_pytorch.deeplabv3.decoder import DeepLabV3PlusDecoder
from segmentation_models_pytorch.fpn.decoder import FPNDecoder
from segmentation_models_pytorch.linknet.decoder import LinknetDecoder
from segmentation_models_pytorch.base import SegmentationHead
from segmentation_models_pytorch.base import initialization as init

class PretrainModel(nn.Module):
    def __init__(
        self,
        encoder_name: str = "timm-efficientnet-b7",
        encoder_weights: Optional[str] = None,
        classes: int = 4,
        in_features: int = 2560, 
        pretrained_path=None, 
        pretrained_num_classes=None,
    ):
        super(PretrainModel, self).__init__()
        self.in_features = in_features
        if pretrained_path is None:
            self.encoder = get_encoder(
                encoder_name,
                in_channels=3,
                depth=5,
                weights=encoder_weights,
            )
            if 'timm-efficientnet' in encoder_name:
                self.hidden_layer = nn.Sequential(*list(self.encoder.children())[-4:])
                del self.encoder.global_pool
                del self.encoder.act2
                del self.encoder.bn2
                del self.encoder.conv_head
            elif 'timm-seresnet' in encoder_name or 'timm-resnet' in encoder_name:
                self.hidden_layer = nn.AdaptiveAvgPool2d(output_size=1)
        else:
            print('Load pretrain: {}'.format(pretrained_path))
            model = PretrainModel(
                encoder_name=encoder_name,
                encoder_weights=encoder_weights,
                classes=pretrained_num_classes,
                in_features=in_features, 
                pretrained_path=None, 
                pretrained_num_classes=None)
            model.load_state_dict(torch.load(pretrained_path, map_location='cpu'))
            self.encoder = model.encoder
            self.hidden_layer = model.hidden_layer
            del model

        self.fc = nn.Linear(in_features, 1024, bias=True)
        self.cls_head = nn.Linear(1024, classes, bias=True)

        init.initialize_head(self.fc)
        init.initialize_head(self.cls_head)

    @autocast(enabled=True)
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.hidden_layer(x)
        x = x.view(-1, self.in_features)
        x = self.fc(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.cls_head(x)
        return x

class SiimCovidAuxModel(nn.Module):
    def __init__(
        self,
        encoder_name: str = "timm-efficientnet-b7",
        encoder_weights: Optional[str] = None,
        decoder_use_batchnorm: bool = True,
        decoder_channels: List[int] = (256, 128, 64, 32, 16),
        decoder_attention_type: Optional[str] = None,
        decoder: Optional[str] = 'unet',
        classes: int = 4,
        in_features: int = 2560,
        encoder_pretrained_path=None, 
        encoder_pretrained_num_classes=None,
        model_pretrained_path=None, 
        model_pretrained_num_classes=None,
        test_mode=False,
    ):
        super(SiimCovidAuxModel, self).__init__()
        self.in_features = in_features
        self.test_mode = test_mode

        if model_pretrained_path is None:
            if encoder_pretrained_path is None:
                model = PretrainModel(
                            encoder_name=encoder_name,
                            encoder_weights=encoder_weights,
                            classes=classes,
                            in_features=in_features, 
                            pretrained_path=None, 
                            pretrained_num_classes=None)
            else:
                print('load pretrain', encoder_pretrained_path)
                model = PretrainModel(
                            encoder_name=encoder_name,
                            encoder_weights=encoder_weights,
                            classes=encoder_pretrained_num_classes,
                            in_features=in_features, 
                            pretrained_path=None, 
                            pretrained_num_classes=None)
                model.load_state_dict(torch.load(encoder_pretrained_path, map_location='cpu'))
            self.encoder = model.encoder
            self.hidden_layer = model.hidden_layer
            del model
            self.fc = nn.Linear(in_features, 1024, bias=True)
            self.cls_head = nn.Linear(1024, classes, bias=True)

            if decoder == 'unet':
                self.decoder = UnetDecoder(
                    encoder_channels=self.encoder.out_channels,
                    decoder_channels=decoder_channels,
                    n_blocks=5,
                    use_batchnorm=decoder_use_batchnorm,
                    center=True if encoder_name.startswith("vgg") else False,
                    attention_type=decoder_attention_type,
                )
            elif decoder == 'unetplusplus':
                self.decoder = UnetPlusPlusDecoder(
                    encoder_channels=self.encoder.out_channels,
                    decoder_channels=decoder_channels,
                    n_blocks=5,
                    use_batchnorm=decoder_use_batchnorm,
                    center=True if encoder_name.startswith("vgg") else False,
                    attention_type=decoder_attention_type,
                )
            elif decoder == 'fpn':
                self.decoder = FPNDecoder(
                    encoder_channels=self.encoder.out_channels,
                    encoder_depth=5,
                    pyramid_channels=256,
                    segmentation_channels=128,
                    dropout=0.2,
                    merge_policy="add",
                )
            elif decoder == 'linknet':
                self.decoder = LinknetDecoder(
                    encoder_channels=self.encoder.out_channels,
                    n_blocks=5,
                    prefinal_channels=32,
                    use_batchnorm=decoder_use_batchnorm,
                )
            elif decoder == 'deeplabv3plus':
                decoder_atrous_rates = [12, 24, 36]
                encoder_output_stride = 16
                self.encoder.make_dilated(
                    stage_list=[5],
                    dilation_list=[2]
                )
                self.decoder = DeepLabV3PlusDecoder(
                    encoder_channels=self.encoder.out_channels,
                    out_channels=decoder_channels,
                    atrous_rates=decoder_atrous_rates,
                    output_stride=encoder_output_stride,
                )
            else:
                raise ValueError('Decoder error!!!')

            if decoder == 'unet' or decoder == 'unetplusplus':
                self.segmentation_head = SegmentationHead(
                    in_channels=decoder_channels[-1],
                    out_channels=1,
                    activation='sigmoid',
                    kernel_size=3,
                )
            elif decoder == 'deeplabv3plus':
                self.segmentation_head = SegmentationHead(
                    in_channels=self.decoder.out_channels,
                    out_channels=1,
                    activation='sigmoid',
                    kernel_size=1,
                    upsampling=4,
                )
            elif decoder == 'fpn':
                self.segmentation_head = SegmentationHead(
                    in_channels=self.decoder.out_channels,
                    out_channels=1,
                    activation='sigmoid',
                    kernel_size=1,
                    upsampling=4,
                )
            elif decoder == 'linknet':
                self.segmentation_head = SegmentationHead(
                    in_channels=32, 
                    out_channels=1, 
                    activation='sigmoid',
                    kernel_size=1
                )
            else:
                raise ValueError('Decoder error!!!')

            init.initialize_head(self.fc)
            init.initialize_head(self.cls_head)
            init.initialize_decoder(self.decoder)
            init.initialize_head(self.segmentation_head)
        else:
            print('Load pretrain: {}'.format(model_pretrained_path))
            model = SiimCovidAuxModel(
                encoder_name=encoder_name,
                encoder_weights=None,
                decoder=decoder,
                classes=model_pretrained_num_classes,
                in_features=in_features,
                decoder_channels=decoder_channels,
                encoder_pretrained_path=None,
                encoder_pretrained_num_classes=None,
                model_pretrained_path=None, 
                model_pretrained_num_classes=None,
                test_mode=False,
            )
            model.load_state_dict(torch.load(model_pretrained_path, map_location='cpu'))

            self.encoder = model.encoder
            self.hidden_layer = model.hidden_layer
            self.decoder = model.decoder
            self.segmentation_head = model.segmentation_head
            self.fc = nn.Linear(in_features, 1024, bias=True)
            self.cls_head = nn.Linear(1024, classes, bias=True)
            del model
            init.initialize_head(self.fc)
            init.initialize_head(self.cls_head)
    
    @autocast(enabled=True)
    def forward(self, x):
        if self.test_mode:
            features = self.encoder(x)
            xcls = self.hidden_layer(features[-1])
            xcls = xcls.view(-1, self.in_features)
            xcls = self.fc(xcls)
            xcls = F.relu(xcls)
            xcls = F.dropout(xcls, p=0.5, training=self.training)
            xcls = self.cls_head(xcls)
            return xcls
        else:
            features = self.encoder(x)
            xseg = self.decoder(*features)
            xseg = self.segmentation_head(xseg)

            xcls = self.hidden_layer(features[-1])
            xcls = xcls.view(-1, self.in_features)
            xcls = self.fc(xcls)
            xcls = F.relu(xcls)
            xcls = F.dropout(xcls, p=0.5, training=self.training)
            xcls = self.cls_head(xcls)

            return xseg, xcls


In [ ]:
class ClfTestDataset(torch.utils.data.Dataset):
    
    def __init__(self, csv, transforms=None):
        self.csv = csv.reset_index()
        self.augmentations = transforms

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        image = cv2.imread(row.test_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']
        
        if('clf_label_idx' in self.csv.columns):
            return torch.tensor(image), torch.tensor(row.clf_label_idx)
        
        return image
    

def get_test_transforms(candidate):
    dim = candidate.get('dim', DIM)
    print(dim)
    return A.Compose(
        [
            A.Resize(dim[0],dim[1],always_apply=True),
            A.Normalize(),
            ToTensorV2(p=1.0)
        ]
    )

In [ ]:
from tqdm.notebook import tqdm 

def clf_predict_fn(dataloader, model, device):
    model.eval()
    tk0 = tqdm(enumerate(dataloader), total=len(dataloader))
    batch_preds=[]
    
    for i, inps in tk0:
        inps = inps.to(device)
        with torch.cuda.amp.autocast(enabled=True):
            logits = model(inps)
            if(torch.isnan(logits.sum())):
                print(torch.isnan(logits.sum(axis=1)).sum())
            sum_check_nan = logits.sum(axis=1)
            logits[torch.isnan(sum_check_nan), :] = 0.25
            
        probs = nn.functional.softmax(logits, dim=-1)
        batch_preds.append(probs.detach().cpu().numpy())
        
        del inps, logits, probs, sum_check_nan
        torch.cuda.empty_cache()
        
    return np.concatenate(batch_preds)

In [ ]:
cfg = dict()
cfg['encoder_name'] = 'timm-efficientnet-b7'
cfg['encoder_weight'] = None
cfg['decoder'] = 'unetplusplus'
cfg['in_features'] = 2560
cfg['decoder_channels'] = [256, 128, 64, 32, 16]
cfg['encoder_pretrained_path'] = None
cfg['model_pretrained_num_classes'] = 4

In [ ]:
for candidate in CANDIDATES:
    cfg['model_pretrained_path'] = candidate.get('model_path')
    model = SiimCovidAuxModel(
    encoder_name=cfg['encoder_name'],
    encoder_weights=cfg['encoder_weight'],
    decoder=cfg['decoder'],
    classes=4,
    in_features=cfg['in_features'],
    decoder_channels=cfg['decoder_channels'],
    encoder_pretrained_path=None ,
    encoder_pretrained_num_classes=None,
    model_pretrained_path=None, 
    model_pretrained_num_classes=cfg['model_pretrained_num_classes'],
    test_mode=True)
    
    model.load_state_dict(torch.load(cfg['model_pretrained_path'], map_location='cpu'))
    model.to('cuda:0')
    print()
    
    batch_size = candidate.get('batch_size', BATCH_SIZE)
    test_dataset = ClfTestDataset(sub_df, get_test_transforms(candidate))
    test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
    
    sub_df[label_cols] += clf_predict_fn(test_dataloader, model, torch.device('cuda:0'))
    
    del model
    torch.cuda.empty_cache()
    
sub_df[label_cols] /= len(CANDIDATES)

In [ ]:
batch_size

In [ ]:
sub_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical', 'test_path']
df = pd.merge(df, sub_df, on = 'id', how = 'left')

# study string

In [ ]:
for i in range(study_len):
    negative = df.loc[i,'negative']
    typical = df.loc[i,'typical']
    indeterminate = df.loc[i,'indeterminate']
    atypical = df.loc[i,'atypical']
    df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'

In [ ]:
df_study = df[['id', 'PredictionString']]

# df.to_csv('submission.csv',index=False)
# df

In [ ]:
sub_df = df_study.fillna('')
def remove_prediction_on_image_level(row):
    if(row['id'].endswith('_image')):
        row['PredictionString'] = ''
    return row
sub_df = sub_df.apply(remove_prediction_on_image_level, axis=1)

In [ ]:
sub_df.iloc[0].PredictionString

In [ ]:
# 'negative 0.01657867431640625 0 0 1 1 typical 0.76240234375 0 0 1 1 indeterminate 0.13795166015625 0 0 1 1 atypical 0.083038330078125 0 0 1 1'

In [ ]:
sub_df.iloc[1].PredictionString

In [ ]:
# 'negative 0.277313232421875 0 0 1 1 typical 0.082476806640625 0 0 1 1 indeterminate 0.1296875 0 0 1 1 atypical 0.510546875 0 0 1 1'

In [ ]:
sub_df.to_csv('submission.csv',index=False)